In [ ]:
import pandas as pd

In [ ]:
# Utility Functions and tests for them

# Get the closing price on the specified date. If there is no entry for that particular date, it will return the value
# for the previous date when the value is available. The date can be specified either as a datetime or as a string. It is 
# assumed that the date and closing value are in a column called Date and Close respectively
def get_closing_on_date(df,date) :
    
    def inner_get_closing_on_date(df, date, min_date, max_date):
        
        #print("date = ", date,min_date,max_date)
        # Sanity Check of range
        if date > max_date or date < min_date :
            assert False, "get_closing_on_date() called with a date that is outside the range"
            
        filtered_df = df[df.Date == date]
        rows = len(filtered_df)
        if rows == 1 :
            # we found a single record with this data
            return filtered_df.iloc[0].Close
        elif rows == 0 :
            new_date = date - pd.DateOffset(days=1)  # try the previous date. Can also use pd.Timedelta(1, unit='d')
            return inner_get_closing_on_date(df, new_date, min_date, max_date)
        else:
            # this should never happened
            assert False, "get_closing_on_date() found multiple records with the same date"
    
    # call the inner function
    min_date = df.Date.min()
    max_date = df.Date.max()
    d = pd.to_datetime(date)
    return inner_get_closing_on_date(df, d, min_date, max_date)

# Test Data ...
test_data = { "Date" : ["2020 Jan 01", "2019 Dec 31", "2019 Dec 30", "2019 Dec 26", "2019 Dec 24"],
        "Close": [100.0, 200.0, 300.0, 400.0, 500.0]}
test_df = pd.DataFrame(test_data)
test_df['Date'] = pd.to_datetime(test_df['Date']) # reformat the date from string to datetime

# Simple tests ....
assert get_closing_on_date(test_df,"2019-12-24") == 500.0, 'Test Failed for 2019-12-24'
assert get_closing_on_date(test_df, pd.to_datetime("2019-12-26")) == 400.0, 'Test Failed for 2019-12-26'

# Tests for dates where data is missing
assert get_closing_on_date(test_df,"2019-12-25") == 500.0, 'Test Failed for 2019-12-25'
assert get_closing_on_date(test_df, pd.to_datetime("2019-12-29")) == 400.0, 'Test Failed for 2019-12-29'

# Tests for dates that are on the max/min of date range
assert get_closing_on_date(test_df,"2019-12-24") == 500.0, 'Test Failed for 2019-12-24'
assert get_closing_on_date(test_df,"2020-01-01") == 100.0, 'Test Failed for 2019-01-01'

# What happens if the input date is earlier than the earliest date ? 
#get_closing_on_date(test_df,"2019-12-20")


In [ ]:
df = pd.read_csv('NIFTY 50_Data.csv',sep=",") 
df['Date'] = pd.to_datetime(df['Date'], format='%d %b %Y') # reformat the date from string to datetime
df.drop( ['Open', 'High', 'Low'], axis=1, inplace=True)  # remove the columns we don't need
df.head()